In [42]:
import numpy
import pandas as pd
import numpy as np
import networkx as nx
from math import ceil

In [43]:
match = 2

# Find Time Points

In [44]:
fullevent_data = pd.read_csv("./2020_Problem_D_DATA/fullevents.csv")

In [45]:
pfed = fullevent_data.loc[:, ['MatchID', 'TeamID', 'MatchPeriod', 'EventTime', 'EventType', 'EventSubType']]
pfed = pfed[pfed['MatchID']==match][pfed['TeamID'].isin(['Huskies'])]

In [46]:
pfed = pfed.reset_index()
pfed = pfed.loc[:, ['MatchID', 'TeamID', 'MatchPeriod', 'EventTime', 'EventType', 'EventSubType']]

In [47]:
last_1H = max(pfed[pfed['MatchPeriod']=='1H'].EventTime)

fed_2H = pfed[pfed['MatchPeriod']=='2H']
fed_2H['EventTime'] += last_1H

pfed = pd.concat([pfed[pfed['MatchPeriod']=='1H'], fed_2H])

In [48]:
points = [0]

points.append(min(pfed[pfed['MatchPeriod']=='2H'].index))

sindex = list(pfed[pfed['EventType']=='Substitution'].index)

for p in sindex:
    points.append(p)

points.append(max(pfed.index))

points = list(set(points))
points = sorted(points)

points

[0, 307, 401, 402, 579, 607]

In [49]:
time_points = []

for i in range(len(points)-1):
    start_time = pfed.iloc[points[i]].EventTime
    end_time = pfed.iloc[points[i+1]].EventTime
    print(start_time, end_time)
    # time_points.append(start_time)

    duration = end_time - start_time
    num_points = ceil(duration / 360)
    if num_points <= 0:
        continue
    sub_quantum = duration / num_points

    while start_time < end_time:
        time_points.append(start_time)
        start_time += sub_quantum

time_points.append(pfed.iloc[points[-1]].EventTime)

34.287353 2798.636123
2798.636123 3758.149957
3758.149957 3758.149957
3758.149957 5498.149957
5498.149957 5751.288263


In [50]:
for i in range(len(time_points)-1):
    print(time_points[i+1]-time_points[i])

345.54359625
345.54359625000006
345.54359625000006
345.54359625000006
345.54359625000006
345.54359624999984
345.54359625000006
345.54359625000006
319.8379446666668
319.8379446666668
319.8379446666663
347.99999999999955
348.0
348.0
348.0
348.0
253.13830600000074


# Handle Passing

In [51]:
passing_data = pd.read_csv("./2020_Problem_D_DATA/passingevents.csv")

In [52]:
passing_data.head()

,MatchID,TeamID,OriginPlayerID,DestinationPlayerID,MatchPeriod,EventTime,EventSubType,EventOrigin_x,EventOrigin_y,EventDestination_x,EventDestination_y
0,1,Huskies,Huskies_D1,Huskies_F1,1H,46.323501,Head pass,34,97,59.0,95.0
1,1,Huskies,Huskies_M1,Huskies_F2,1H,51.022546,Simple pass,53,89,69.0,91.0
2,1,Opponent1,Opponent1_D2,Opponent1_G1,1H,89.008721,Simple pass,19,16,5.0,50.0
3,1,Opponent1,Opponent1_G1,Opponent1_F1,1H,92.216160,Launch,5,50,67.0,44.0
4,1,Huskies,Huskies_M2,Huskies_M3,1H,98.265191,Simple pass,42,55,36.0,54.0


In [53]:
PPD = passing_data.loc[:, ['MatchID', 'TeamID', 'OriginPlayerID', 'DestinationPlayerID', 'EventTime', 'EventOrigin_x', 'EventOrigin_y']]
PPD = PPD[PPD['MatchID']==match][PPD['TeamID'].isin(['Huskies'])]

# Centroid coordinates

In [55]:
ccs = []

for i in range(len(time_points)-1):
    start_time = time_points[i]
    end_time = time_points[i+1]
    if i == len(time_points)-2:
        end_time += 360

    ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]
    
    X_mean, Y_mean = ppd[['EventOrigin_x', 'EventOrigin_y']].mean()
    # pomd = ppd.groupby('OriginPlayerID').mean()
    # pomd['distance'] = [((x - X_mean)**2 + (y - Y_mean)**2)**0.5 for x,y in zip(pomd['EventOrigin_x'],pomd['EventOrigin_y'])]

    ccs.append((X_mean, Y_mean))

centroid_coordinates_and_dispersion = pd.DataFrame({'Centroid coordinates':ccs})

centroid_coordinates_and_dispersion.to_csv("./results/short/match_{}/centroid_coordinates_and_dispersion.csv".format(match))

# Clustering coefficient

In [56]:
for i in range(len(time_points)-1):
    start_time = time_points[i]
    end_time = time_points[i+1]
    if i == len(time_points)-2:
        end_time += 360

    ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]
    
    ppd['count'] = 1
    ppd = ppd.groupby(['OriginPlayerID', 'DestinationPlayerID']).sum()
    ppd = ppd.sort_values(by='count').ix[::-1]

    temple_ppd = ppd.copy()
    if len(temple_ppd)==0:
        continue
    max_count = max(temple_ppd['count'])
    temple_ppd['count'] = temple_ppd['count'] / max_count
    
    d0 = temple_ppd.copy()
    d0 = pd.DataFrame({'i': list(map(lambda x: x[0], d0.index)), 'j': list(map(lambda x: x[1], d0.index)), 'count':d0.to_numpy().T[0]})
    d1 = d0.copy()
    d2 = d0.copy()

    d1 = d1.rename(columns={'i':'j', "j":'k'})
    d2 = d2.rename(columns={'i':'k', "j":'i'})

    merged_data = pd.merge(pd.merge(d0, d1, on='j'), d2, on=['k', 'i'])
    md = merged_data

    md = md.rename(columns={'count_x':'wij', 'count_y':'wjk', 'count':'wki'})

    md['www'] = [(wij * wjk * wki) for wij, wjk, wki in zip(md['wij'], md['wjk'], md['wki'])]
    md['ww'] = [(wij * wki) for wij, wki in zip(md['wij'], md['wki'])]

    md = md.groupby('i').sum()

    md['Clustering coefficient'] = [www/ww for www,ww in zip(md['www'], md['ww'])]
    md = md.sort_values(by='Clustering coefficient').ix[::-1]

    md.to_csv('./results/short/match_{}/{}-Clustering coefficient.csv'.format(match, i))

# Eigenvector centrality dispersion

In [57]:
dispersions = []

for i in range(len(time_points)-1):
    start_time = time_points[i]
    end_time = time_points[i+1]
    if i == len(time_points)-2:
        end_time += 360

    ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]

    ppd = ppd.rename(columns={'OriginPlayerID':'i', 'DestinationPlayerID':'j'})
    
    players = list(set(ppd['i']))
    players = sorted(players)

    mat = np.zeros((30, 30))

    for i, iplayer in enumerate(players):
        for j, jplayer in enumerate(players):
            qr = ppd[ppd['i']==iplayer][ppd['j']==jplayer].to_numpy()
            if len(qr) > 0:
                mat[i][j] = qr[-1][-1]
            else:
                mat[i][j] = 0
    
    A = mat

    G = nx.Graph()

    for i, p in enumerate(players):
        G.add_node(i)

    for i in range(len(players)):
        for j in range(len(players)):
            G.add_edge(i, j, weight=A[i][j])

    try:
        ec = nx.eigenvector_centrality_numpy(G, weight='weight')
    except:
        continue

    ps = []
    vs = []
    for p, v in ec.items():
        ps.append(p)
        vs.append(v)

    max_ec = max(vs)
    player_with_ec = ps[vs.index(max_ec)]

    dispersion_ec = np.var(vs)

    dispersions.append(dispersion_ec)

pd.DataFrame({'dispersion':dispersions}).to_csv('./results/short/match_{}/Eigenvector centrality dispersion.csv'.format(match))

In [58]:
A

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0